##### Dataset Load

In [5]:
import pandas as pd
null_values = ['?', '??', 'N/A', 'NA', 'nan', 'NaN', '-nan', '-NaN', 'null', '-']
x_test = pd.read_csv('./data/x_test_normal.csv', na_values = null_values)
y_test = pd.read_csv('./data/y_test_normal.csv', na_values = null_values)
x_test_features = x_test.drop(columns=['날짜', 'CODE', '종가'], inplace=False)
y_test_bool = y_test['Y'] <-2.0

### 1. Tree Base Simple Classifers

##### 1-1 Decision Tree Model

In [4]:
import pickle
import joblib
from sklearn import tree
from sklearn.metrics import classification_report

decisionTree = tree.DecisionTreeClassifier(
    max_depth=15,
    min_samples_split=100,
    class_weight={True: 10, False: 1}
)

decisionTree = joblib.load('./models/decisionTree.pkl') 
y_pred = decisionTree.predict(x_test_features)
target_names = ['no risk', 'risk']
print(classification_report(y_test_bool, y_pred, target_names = target_names))

              precision    recall  f1-score   support

     no risk       0.90      0.31      0.46     21040
        risk       0.20      0.83      0.32      4311

    accuracy                           0.40     25351
   macro avg       0.55      0.57      0.39     25351
weighted avg       0.78      0.40      0.43     25351



##### 1-2 Random Forest

In [8]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200, 
    criterion='entropy', 
    min_samples_split = 100,
    bootstrap=True,
    max_depth=20,
    class_weight={True: 10, False: 1}
    )
rf = joblib.load('./models/randomForest.pkl') 
y_pred = rf.predict(x_test_features)
target_names = ['no risk', 'risk']
print(classification_report(y_test_bool, y_pred, target_names = target_names))

              precision    recall  f1-score   support

     no risk       0.91      0.38      0.54     21040
        risk       0.21      0.82      0.34      4311

    accuracy                           0.46     25351
   macro avg       0.56      0.60      0.44     25351
weighted avg       0.79      0.46      0.50     25351



### 2. LightGBM and Weak Bagging